# 📊 01 - Limpieza de Datos

**TFM: Predicción de Abandono Universitario**

Este notebook procesa los Excel originales y genera las tablas limpias en formato .parquet

**Autora:** María José Morte (morte@uji.es)

## 1. Configuración Inicial

In [1]:
# =============================================================================
# CONFIGURACIÓN INICIAL
# =============================================================================

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurar rutas
import sys
sys.path.append('../src')

try:
    from config import DATA_RAW, DATA_INTERIM, info_entorno
    info_entorno()
except ImportError:
    # Fallback si no encuentra config
    DATA_RAW = Path('../data/01_raw')
    DATA_INTERIM = Path('../data/02_interim')
    print(f"Usando rutas por defecto")

# Crear carpeta interim si no existe
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Configuración cargada")

INFORMACIÓN DEL ENTORNO
Entorno: local
Raíz proyecto: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_
Data RAW: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\data\01_raw
Data INTERIM: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\data\02_interim
Data PROCESSED: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\data\03_processed
Docs: C:\Users\mjmor\0.-TFM\TFM_abandono_fase1_\docs

✅ Configuración cargada


## 2. Funciones de Transformación

In [2]:
# =============================================================================
# FUNCIONES DE TRANSFORMACIÓN
# =============================================================================

def normalizar_nota_selectividad(valor):
    """Normaliza nota selectividad de formato 525 a 5.25"""
    if pd.isna(valor):
        return np.nan
    try:
        valor = float(valor)
        if valor > 100:  # Formato 525, 11385, etc.
            if valor > 10000:
                return valor / 1000  # 11385 -> 11.385
            else:
                return valor / 100   # 525 -> 5.25
        return valor
    except:
        return np.nan

def corregir_cp(cp):
    """Corrige códigos postales de 4 dígitos añadiendo 0 al principio"""
    if pd.isna(cp):
        return None
    cp_str = str(int(cp)).strip()
    if len(cp_str) == 4:
        return '0' + cp_str
    return cp_str

def guardar_parquet(df, nombre, carpeta=DATA_INTERIM):
    """Guarda DataFrame en parquet y muestra resumen"""
    path = carpeta / f'{nombre}_limpio.parquet'
    df.to_parquet(path, index=False)
    print(f"\n✅ Guardado: {path.name}")
    print(f"   Registros: {len(df):,}")
    print(f"   Columnas: {list(df.columns)}")
    return path

print("✅ Funciones de transformación definidas")

✅ Funciones de transformación definidas


## 3. Diccionarios de Traducción

In [3]:
# =============================================================================
# DICCIONARIOS DE TRADUCCIÓN
# =============================================================================

# Cupos de acceso
TRAD_CUPO = {
    'General': 'General',
    'Mayor 25 A±os': 'Mayor 25 años',
    'Mayor 25 Años': 'Mayor 25 años',
    'Mayor 40 A±os': 'Mayor 40 años',
    'Mayor 40 Años': 'Mayor 40 años',
    'Mayor 45 A±os': 'Mayor 45 años',
    'Mayor 45 Años': 'Mayor 45 años',
    'Minusvalidos': 'Diversidad funcional',
    'Minusválidos': 'Diversidad funcional',
    'Deportistas': 'Deportistas',
    'Titulados': 'Titulados',
    'Extranjeros': 'Extranjeros'
}

# Estados de preinscripción
TRAD_ESTADO = {
    'AS': 'Asignada',
    'LE': 'Lista espera',
    'SO': 'Solicitada',
    '*': 'Desconocido'
}

# Ramas de conocimiento
TRAD_RAMA = {
    'SO': 'Ciencias Sociales y Jurídicas',
    'HU': 'Humanidades',
    'EX': 'Ciencias Experimentales',
    'TE': 'Técnicas e Ingeniería',
    'SA': 'Ciencias de la Salud'
}

# Becas (valenciano -> castellano)
TRAD_BECAS = {
    'Becari': 'Becario',
    'Beca denegada': 'Beca denegada',
    "Alumne d'intercanvi": 'Alumno de intercambio',
    'Becari total, no paga cap crèdit': 'Becario total (no paga créditos)',
    'Beca GVA denegada': 'Beca GVA denegada',
    'Temps parcial obligat': 'Tiempo parcial obligatorio',
    "Grup d'alt rendiment acadèmic": 'Grupo alto rendimiento académico',
    'Exent del requisits de matrícula': 'Exento requisitos matrícula',
    'Temps parcial': 'Tiempo parcial',
    'Becari (assignatures de segona matrícula)': 'Becario (2ª matrícula)',
    'Becari GVA': 'Becario GVA',
    'Becari (3a matrícula)': 'Becario (3ª matrícula)',
    'Becari (4a matrícula)': 'Becario (4ª matrícula)',
    'Doble Titulació Internacional': 'Doble Titulación Internacional',
    'Incompleix normativa de rendiment': 'Incumple normativa rendimiento',
    'Incompleix normativa de permanència': 'Incumple normativa permanencia'
}

# Trabajo (valenciano -> castellano)
TRAD_TRABAJO = {
    'Inactiu o desocupat': 'Inactivo o desempleado',
    'Empleats de tipus administratiu': 'Empleados administrativos',
    'Treballadors no qualificats': 'Trabajadores no cualificados',
    'Treballadors dels serveis de restauració, personals, protecció i venedors dels comerços': 'Servicios y comercio',
    'Tècnics i Professionals científics i intel·lectuals': 'Técnicos y profesionales científicos',
    "Direcció d'Empreses i de les Administracions Públiques": 'Dirección empresas y administración pública',
    'Tècnics i Professionals de recolzament': 'Técnicos y profesionales de apoyo',
    "Artesans i treballadors qualificats de les indústries manufactureres, la construcció i la mineria, excepte els operadors d'instal·lacions i maquinària": 'Industria y construcción',
    "Treballadors qualificats en l'agricultura i la pesca": 'Agricultura y pesca',
    "Operadors d'instal·lacions i maquinaria i muntadors": 'Operadores maquinaria',
    'Forces Armades': 'Fuerzas Armadas',
    'No realitza cap treball remunerat': 'Sin trabajo remunerado'
}

# Forma de pago
TRAD_FORMA_PAGO = {
    'D': 'Domiciliado',
    'N': 'Recibo',
    'T': 'Tarjeta'
}

print("✅ Diccionarios de traducción definidos")

✅ Diccionarios de traducción definidos


## 4. Procesar PREINSCRIPCION

In [4]:
# =============================================================================
# PROCESAR PREINSCRIPCION
# =============================================================================
print("="*60)
print("PROCESANDO: PREINSCRIPCION")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'preinscripcion_si.xlsx')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar y renombrar columnas
df_limpio = df[['Per_id_ficticio', 'ANO', 'UNIVERSIDAD', 'MUNICIPIO', 'CP', 
                'ORDEN_TITULACION', 'NOM_CUPO', 'ESTADO', 'NOTA_TXT', 'ESTUDIO']].copy()

df_limpio.columns = ['per_id_ficticio', 'ano', 'universidad', 'municipio_preins', 'cp_preins',
                     'orden_preferencia', 'cupo', 'estado', 'nota_selectividad', 'titulacion_nombre']

# Transformaciones
df_limpio['per_id_ficticio'] = df_limpio['per_id_ficticio'].astype('int32')
df_limpio['ano'] = df_limpio['ano'].astype('int16')
df_limpio['orden_preferencia'] = df_limpio['orden_preferencia'].astype('int8')
df_limpio['universidad'] = df_limpio['universidad'].astype('category')
df_limpio['municipio_preins'] = df_limpio['municipio_preins'].str.strip().str.title()
df_limpio['cp_preins'] = df_limpio['cp_preins'].apply(corregir_cp)
df_limpio['cupo'] = df_limpio['cupo'].map(TRAD_CUPO).fillna(df_limpio['cupo']).astype('category')
df_limpio['estado'] = df_limpio['estado'].map(TRAD_ESTADO).fillna('Desconocido').astype('category')
df_limpio['nota_selectividad'] = df_limpio['nota_selectividad'].apply(normalizar_nota_selectividad).astype('float32')
df_limpio['titulacion_nombre'] = df_limpio['titulacion_nombre'].str.strip().astype('category')

# Guardar
guardar_parquet(df_limpio, 'preinscripcion')

PROCESANDO: PREINSCRIPCION
Registros originales: 210,996
Columnas originales: 25

✅ Guardado: preinscripcion_limpio.parquet
   Registros: 210,996
   Columnas: ['per_id_ficticio', 'ano', 'universidad', 'municipio_preins', 'cp_preins', 'orden_preferencia', 'cupo', 'estado', 'nota_selectividad', 'titulacion_nombre']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/preinscripcion_limpio.parquet')

In [5]:
# Ver nombres de columnas reales en Expedientes
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Expedientes')
print("Columnas en Expedientes:")
for col in df.columns:
    print(f"  '{col}'")

Columnas en Expedientes:
  'Per_id_ficticio'
  'Exp_Tit_ Id'
  'Curso Aca Ini'
  'Curso Aca'
  'Curso Aca Fin'
  'Nota'
  'Nombre'
  'Seguro'
  'Nau(Nota selectividad)'
  'Nota Acceso'
  'Cred Matriculados'
  'Cred_Superados'
  'Egresado'
  'Nuevo'
  'Media_Curso'


In [6]:
# Ver valores problemáticos
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Expedientes')

print("Valores únicos en Seguro:", df['Seguro'].unique()[:20])
print("Valores únicos en Egresado:", df['Egresado'].unique()[:20])
print("Valores únicos en Nuevo:", df['Nuevo'].unique()[:20])

Valores únicos en Seguro: ['N' 'S']
Valores únicos en Egresado: ['N' 'S']
Valores únicos en Nuevo: ['N' 'S']


In [7]:
# Ver valores problemáticos en columnas numéricas
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Expedientes')

print("Curso Aca Fin - valores únicos:")
print(df['Curso Aca Fin'].unique()[:20])

print("\nNota - valores únicos:")
print(df['Nota'].unique()[:20])

Curso Aca Fin - valores únicos:
[2014 '-' 2013 2011 2018 2010 2012 2019 2017 2016 2015 2020]

Nota - valores únicos:
[8.68 '-' 8.01 6.1000000000000005 6.83 6.34 6.23 6.390000000000001
 6.2700000000000005 9.370000000000001 8 6.38 7.55 7.390000000000001 7.63
 6.44 6.98 7.15 6.54 8.39]


## 5. Procesar EXPEDIENTES

In [8]:
# =============================================================================
# PROCESAR EXPEDIENTES
# =============================================================================
print("="*60)
print("PROCESANDO: EXPEDIENTES")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Expedientes')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar columnas (nombres reales del Excel)
cols_usar = ['Per_id_ficticio', 'Exp_Tit_ Id', 'Curso Aca', 'Curso Aca Fin',
             'Nota', 'Nombre', 'Seguro', 'Nau(Nota selectividad)', 
             'Nota Acceso', 'Cred Matriculados', 'Cred_Superados', 
             'Egresado', 'Nuevo', 'Media_Curso']

df_limpio = df[cols_usar].copy()

df_limpio.columns = ['per_id_ficticio', 'exp_tit_id', 'curso_aca', 'curso_aca_fin',
                     'nota_1', 'via_acceso', 'seguro', 'nota_selectividad_exp',
                     'nota_acceso_exp', 'cred_matriculados', 'cred_superados',
                     'egresado', 'nuevo', 'media_curso']

# Reemplazar '-' por NaN (valores vacíos)
df_limpio = df_limpio.replace('-', np.nan)

# Transformaciones
df_limpio['per_id_ficticio'] = df_limpio['per_id_ficticio'].astype('int32')
df_limpio['exp_tit_id'] = df_limpio['exp_tit_id'].astype('int16')
df_limpio['curso_aca'] = df_limpio['curso_aca'].astype('int16')
df_limpio['curso_aca_fin'] = pd.to_numeric(df_limpio['curso_aca_fin'], errors='coerce').astype('Int16')
df_limpio['nota_1'] = pd.to_numeric(df_limpio['nota_1'], errors='coerce').astype('float32')
df_limpio['via_acceso'] = df_limpio['via_acceso'].astype('category')
df_limpio['seguro'] = df_limpio['seguro'].map({'S': 1, 'N': 0}).astype('int8')
df_limpio['egresado'] = df_limpio['egresado'].map({'S': 1, 'N': 0}).astype('int8')
df_limpio['nuevo'] = df_limpio['nuevo'].map({'S': 1, 'N': 0}).astype('int8')
df_limpio['nota_selectividad_exp'] = pd.to_numeric(df_limpio['nota_selectividad_exp'], errors='coerce').astype('float32')
df_limpio['nota_acceso_exp'] = pd.to_numeric(df_limpio['nota_acceso_exp'], errors='coerce').astype('float32')
df_limpio['cred_matriculados'] = pd.to_numeric(df_limpio['cred_matriculados'], errors='coerce').astype('float32')
df_limpio['cred_superados'] = pd.to_numeric(df_limpio['cred_superados'], errors='coerce').astype('float32')
df_limpio['media_curso'] = pd.to_numeric(df_limpio['media_curso'], errors='coerce').astype('float32')

# Guardar
guardar_parquet(df_limpio, 'expedientes')

PROCESANDO: EXPEDIENTES
Registros originales: 109,575
Columnas originales: 15

✅ Guardado: expedientes_limpio.parquet
   Registros: 109,575
   Columnas: ['per_id_ficticio', 'exp_tit_id', 'curso_aca', 'curso_aca_fin', 'nota_1', 'via_acceso', 'seguro', 'nota_selectividad_exp', 'nota_acceso_exp', 'cred_matriculados', 'cred_superados', 'egresado', 'nuevo', 'media_curso']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/expedientes_limpio.parquet')

In [9]:
# Ver columnas reales en Titulaciones
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Titulaciones')
print("Columnas en Titulaciones:")
for col in df.columns:
    print(f"  '{col}'")

Columnas en Titulaciones:
  'Exp_Tit_ Id'
  'Titulación'
  'Rama'
  'Créd_Titulación'
  'Tipo'
  'Tipo Estudio'
  'Unnamed: 6'
  'Unnamed: 7'
  'Unnamed: 8'
  'Unnamed: 9'


## 6. Procesar TITULACIONES

In [10]:
# =============================================================================
# PROCESAR TITULACIONES
# =============================================================================
print("="*60)
print("PROCESANDO: TITULACIONES")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Titulaciones')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar columnas (con tildes)
df_limpio = df[['Exp_Tit_ Id', 'Titulación', 'Rama', 'Créd_Titulación']].copy()

df_limpio.columns = ['exp_tit_id', 'titulacion_nombre', 'rama', 'cred_titulacion']

# Eliminar filas con valores vacíos (son filas basura del Excel)
df_limpio = df_limpio.dropna(subset=['exp_tit_id'])

# Transformaciones
df_limpio['exp_tit_id'] = df_limpio['exp_tit_id'].astype('int16')
df_limpio['titulacion_nombre'] = df_limpio['titulacion_nombre'].str.strip().astype('category')
df_limpio['rama'] = df_limpio['rama'].map(TRAD_RAMA).fillna(df_limpio['rama']).astype('category')
df_limpio['cred_titulacion'] = pd.to_numeric(df_limpio['cred_titulacion'], errors='coerce').astype('Int16')

# Guardar
guardar_parquet(df_limpio, 'titulaciones')

PROCESANDO: TITULACIONES
Registros originales: 108
Columnas originales: 10

✅ Guardado: titulaciones_limpio.parquet
   Registros: 45
   Columnas: ['exp_tit_id', 'titulacion_nombre', 'rama', 'cred_titulacion']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/titulaciones_limpio.parquet')

## 7. Procesar NAC_SEXO

In [11]:
# =============================================================================
# PROCESAR NAC_SEXO
# =============================================================================
print("="*60)
print("PROCESANDO: NAC_SEXO")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Nac-Sexo_Nacionalidad')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar columnas
df_limpio = df[['Per_id_ficticio', 'Sexo', 'Fecha nacimiento', 'Pais Nombre']].copy()

df_limpio.columns = ['per_id_ficticio', 'sexo', 'fecha_nacimiento', 'pais']

# Transformaciones
df_limpio['per_id_ficticio'] = df_limpio['per_id_ficticio'].astype('int32')
df_limpio['sexo'] = df_limpio['sexo'].astype('int8')  # 1=Hombre, 2=Mujer

# Calcular edad (año 2021 como referencia)
df_limpio['edad'] = 2021 - pd.to_datetime(df_limpio['fecha_nacimiento']).dt.year
df_limpio['edad'] = df_limpio['edad'].astype('int8')
df_limpio = df_limpio.drop('fecha_nacimiento', axis=1)

df_limpio['pais'] = df_limpio['pais'].str.strip().astype('category')

# Guardar
guardar_parquet(df_limpio, 'nac_sexo')

PROCESANDO: NAC_SEXO
Registros originales: 30,873
Columnas originales: 5

✅ Guardado: nac_sexo_limpio.parquet
   Registros: 30,873
   Columnas: ['per_id_ficticio', 'sexo', 'pais', 'edad']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/nac_sexo_limpio.parquet')

## 8. Procesar DOMICILIOS

In [12]:
# Ver columnas reales en Domicilios
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Domicilios')
print("Columnas en Domicilios:")
for col in df.columns:
    print(f"  '{col}'")

Columnas en Domicilios:
  'Per_id_ficticio'
  'Población'
  'Provincia'
  'País'
  'Curso Aca'
  'Tipo Domicilio'


In [13]:
# =============================================================================
# PROCESAR DOMICILIOS
# =============================================================================
print("="*60)
print("PROCESANDO: DOMICILIOS")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Domicilios')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar columnas (con tildes)
df_limpio = df[['Per_id_ficticio', 'Curso Aca', 'Población', 'Provincia', 'País', 'Tipo Domicilio']].copy()

df_limpio.columns = ['per_id_ficticio', 'curso_aca', 'poblacion', 'provincia', 'pais_domicilio', 'tipo_domicilio']

# Reemplazar '-' por NaN
df_limpio = df_limpio.replace('-', np.nan)

# Transformaciones
df_limpio['per_id_ficticio'] = df_limpio['per_id_ficticio'].astype('int32')
df_limpio['curso_aca'] = df_limpio['curso_aca'].astype('int16')
df_limpio['poblacion'] = df_limpio['poblacion'].str.strip()
df_limpio['provincia'] = df_limpio['provincia'].str.strip().astype('category')
df_limpio['pais_domicilio'] = df_limpio['pais_domicilio'].str.strip().astype('category')
df_limpio['tipo_domicilio'] = df_limpio['tipo_domicilio'].str.strip().astype('category')

# Seleccionar 1 domicilio por alumno+año (priorizar 'Durante el curso')
def seleccionar_domicilio(grupo):
    if len(grupo) == 1:
        return grupo.iloc[0]
    durante_curso = grupo[grupo['tipo_domicilio'].str.contains('Durante', case=False, na=False)]
    if len(durante_curso) > 0:
        return durante_curso.iloc[0]
    return grupo.iloc[0]

df_limpio = df_limpio.groupby(['per_id_ficticio', 'curso_aca'], as_index=False).apply(
    seleccionar_domicilio
).reset_index(drop=True)

# Guardar
guardar_parquet(df_limpio, 'domicilios')

PROCESANDO: DOMICILIOS
Registros originales: 210,911
Columnas originales: 6

✅ Guardado: domicilios_limpio.parquet
   Registros: 109,206
   Columnas: ['per_id_ficticio', 'curso_aca', 'poblacion', 'provincia', 'pais_domicilio', 'tipo_domicilio']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/domicilios_limpio.parquet')

## 9. Procesar BECAS

In [14]:
# =============================================================================
# PROCESAR BECAS
# =============================================================================
print("="*60)
print("PROCESANDO: BECAS")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Circunstancias')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar columnas
df_limpio = df[['Per_id_Ficticio', 'Mat Curso Aca', 'Nombre Ca']].copy()

df_limpio.columns = ['per_id_ficticio', 'curso_aca', 'beca']

# Transformaciones
df_limpio['per_id_ficticio'] = df_limpio['per_id_ficticio'].astype('int32')
df_limpio['curso_aca'] = df_limpio['curso_aca'].astype('int16')
df_limpio['beca'] = df_limpio['beca'].map(TRAD_BECAS).fillna(df_limpio['beca']).astype('category')

# Guardar
guardar_parquet(df_limpio, 'becas')

PROCESANDO: BECAS
Registros originales: 70,524
Columnas originales: 4

✅ Guardado: becas_limpio.parquet
   Registros: 70,524
   Columnas: ['per_id_ficticio', 'curso_aca', 'beca']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/becas_limpio.parquet')

## 10. Procesar TRABAJO

In [15]:
# =============================================================================
# PROCESAR TRABAJO
# =============================================================================
print("="*60)
print("PROCESANDO: TRABAJO")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Trabajo')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar columnas (sin Exp_Tit_ Id que tiene datos erróneos)
df_limpio = df[['Per_id_ficticio', 'Mat Curso Aca', 'Nombre_trabajo']].copy()

df_limpio.columns = ['per_id_ficticio', 'curso_aca', 'trabajo']

# Transformaciones
df_limpio['per_id_ficticio'] = df_limpio['per_id_ficticio'].astype('int32')
df_limpio['curso_aca'] = df_limpio['curso_aca'].astype('int16')
df_limpio['trabajo'] = df_limpio['trabajo'].map(TRAD_TRABAJO).fillna(df_limpio['trabajo']).astype('category')

# Guardar
guardar_parquet(df_limpio, 'trabajo')

PROCESANDO: TRABAJO
Registros originales: 195,524
Columnas originales: 4

✅ Guardado: trabajo_limpio.parquet
   Registros: 195,524
   Columnas: ['per_id_ficticio', 'curso_aca', 'trabajo']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/trabajo_limpio.parquet')

## 11. Procesar NOTAS

In [16]:
# =============================================================================
# PROCESAR NOTAS
# =============================================================================
print("="*60)
print("PROCESANDO: NOTAS")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Notas')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar columnas (sin Titu que tiene datos erróneos)
df_limpio = df[['Per_id_ficticio', 'Curso Aca', 'Media Titulacion curso', 'Media Titulacion Curso Per_id']].copy()

df_limpio.columns = ['per_id_ficticio', 'curso_aca', 'media_titulacion_curso', 'media_alumno_curso']

# Transformaciones
df_limpio['per_id_ficticio'] = df_limpio['per_id_ficticio'].astype('int32')
df_limpio['curso_aca'] = df_limpio['curso_aca'].astype('int16')
df_limpio['media_titulacion_curso'] = df_limpio['media_titulacion_curso'].astype('float32')
df_limpio['media_alumno_curso'] = df_limpio['media_alumno_curso'].astype('float32')

# Guardar
guardar_parquet(df_limpio, 'notas')

PROCESANDO: NOTAS
Registros originales: 107,908
Columnas originales: 5

✅ Guardado: notas_limpio.parquet
   Registros: 107,908
   Columnas: ['per_id_ficticio', 'curso_aca', 'media_titulacion_curso', 'media_alumno_curso']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/notas_limpio.parquet')

## 12. Procesar RECIBOS

In [17]:
# =============================================================================
# PROCESAR RECIBOS
# =============================================================================
print("="*60)
print("PROCESANDO: RECIBOS")
print("="*60)

# Leer Excel
df = pd.read_excel(DATA_RAW / 'datos_proyecto_sin_preinscrip.xlsx', sheet_name='Recibos')
print(f"Registros originales: {len(df):,}")
print(f"Columnas originales: {len(df.columns)}")

# Seleccionar columnas
df_limpio = df[['Per_id_ficticio', 'Curso Aca', 'Forma De Pago', 'Numero Pagos']].copy()

df_limpio.columns = ['per_id_ficticio', 'curso_aca', 'forma_pago', 'numero_pagos']

# Transformaciones
df_limpio['per_id_ficticio'] = df_limpio['per_id_ficticio'].astype('int32')
df_limpio['curso_aca'] = df_limpio['curso_aca'].astype('int16')
df_limpio['forma_pago'] = df_limpio['forma_pago'].map(TRAD_FORMA_PAGO).fillna(df_limpio['forma_pago']).astype('category')
df_limpio['numero_pagos'] = df_limpio['numero_pagos'].astype('int8')

# Guardar
guardar_parquet(df_limpio, 'recibos')

PROCESANDO: RECIBOS
Registros originales: 114,447
Columnas originales: 5

✅ Guardado: recibos_limpio.parquet
   Registros: 114,447
   Columnas: ['per_id_ficticio', 'curso_aca', 'forma_pago', 'numero_pagos']


WindowsPath('C:/Users/mjmor/0.-TFM/TFM_abandono_fase1_/data/02_interim/recibos_limpio.parquet')

## 13. Resumen Final

In [18]:
# =============================================================================
# RESUMEN FINAL
# =============================================================================
print("\n" + "="*60)
print("RESUMEN FINAL - FASE 1: LIMPIEZA DE DATOS")
print("="*60)

# Listar ficheros generados
print("\n📁 FICHEROS GENERADOS:")
total_registros = 0
total_columnas = 0

for f in sorted(DATA_INTERIM.glob('*.parquet')):
    df = pd.read_parquet(f)
    total_registros += len(df)
    total_columnas += len(df.columns)
    print(f"  {f.name}: {len(df):,} registros, {len(df.columns)} columnas")

print(f"\n📊 TOTALES:")
print(f"  Tablas: {len(list(DATA_INTERIM.glob('*.parquet')))}")
print(f"  Registros: {total_registros:,}")
print(f"  Columnas (suma): {total_columnas}")

print("\n" + "="*60)
print("✅ FASE 1 COMPLETADA")
print("Siguiente paso: Ejecutar 02_genera_reportes_sweetviz_dinamico.ipynb")
print("="*60)


RESUMEN FINAL - FASE 1: LIMPIEZA DE DATOS

📁 FICHEROS GENERADOS:
  becas_limpio.parquet: 70,524 registros, 3 columnas
  domicilios_limpio.parquet: 109,206 registros, 6 columnas
  expedientes_limpio.parquet: 109,575 registros, 14 columnas
  nac_sexo_limpio.parquet: 30,873 registros, 4 columnas
  notas_limpio.parquet: 107,908 registros, 4 columnas
  preinscripcion_limpio.parquet: 210,996 registros, 10 columnas
  recibos_limpio.parquet: 114,447 registros, 4 columnas
  titulaciones_limpio.parquet: 45 registros, 4 columnas
  trabajo_limpio.parquet: 195,524 registros, 3 columnas

📊 TOTALES:
  Tablas: 9
  Registros: 949,098
  Columnas (suma): 52

✅ FASE 1 COMPLETADA
Siguiente paso: Ejecutar 02_genera_reportes_sweetviz_dinamico.ipynb
